In [0]:
df = spark.sql("select * from pyspark_cata.source.products")

In [0]:
#removing duplicates

from pyspark.sql.window import Window
from pyspark.sql.functions import *

w = Window.partitionBy("id").orderBy(desc("updatedDate"))
df = df.withColumn("rn",row_number().over(w))

df = df.filter(col("rn")==1).drop(col("rn"))


In [0]:
from delta.tables import DeltaTable

if DeltaTable.isDeltaTable(spark, "/Volumes/pyspark_cata/source/db_volume/products_sink/"):
    
    dt = DeltaTable.forPath(spark,"/Volumes/pyspark_cata/source/db_volume/products_sink/")
    dt.alias("tgt").merge(df.alias("src"),"tgt.id=src.id")\
    .whenMatchedUpdateAll(condition="src.updatedDate>=tgt.updatedDate")\
        .whenNotMatchedInsertAll().execute()

else:

    df.write.format("delta").mode("Overwrite").save("/Volumes/pyspark_cata/source/db_volume/products_sink/")


In [0]:
%sql
SELECT * FROM delta.`/Volumes/pyspark_cata/source/db_volume/products_sink/`